In [1]:
import pandas as pd


In [5]:
small_set_data = pd.read_parquet("data/Zzzs_train.parquet")

In [6]:
print(small_set_data)

             series_id    step                 timestamp     anglez    enmo  \
0         08db4255286f       0  2018-11-05T10:00:00-0400 -30.845301  0.0447   
1         08db4255286f       1  2018-11-05T10:00:05-0400 -34.181801  0.0443   
2         08db4255286f       2  2018-11-05T10:00:10-0400 -33.877102  0.0483   
3         08db4255286f       3  2018-11-05T10:00:15-0400 -34.282101  0.0680   
4         08db4255286f       4  2018-11-05T10:00:20-0400 -34.385799  0.0768   
...                ...     ...                       ...        ...     ...   
13165555  d5e47b94477e  395815  2017-12-02T15:44:35-0500 -10.064100  0.1552   
13165556  d5e47b94477e  395816  2017-12-02T15:44:40-0500 -10.006700  0.1139   
13165557  d5e47b94477e  395817  2017-12-02T15:44:45-0500  -3.328600  0.1245   
13165558  d5e47b94477e  395818  2017-12-02T15:44:50-0500  -5.903100  0.1363   
13165559  d5e47b94477e  395819  2017-12-02T15:44:55-0500 -11.691100  0.0995   

          awake  
0             1  
1             1

In [7]:
data_series = {}
for series in set(small_set_data["series_id"]):
    data_series[series] = small_set_data[small_set_data["series_id"] == series]

In [17]:
print(data_series['18b61dd5aae8'].iloc[0:10])

            series_id  step                 timestamp     anglez    enmo  \
1885140  18b61dd5aae8     0  2017-12-22T16:30:00-0500 -14.944400  0.0091   
1885141  18b61dd5aae8     1  2017-12-22T16:30:05-0500 -15.538300  0.0204   
1885142  18b61dd5aae8     2  2017-12-22T16:30:10-0500 -15.362400  0.0048   
1885143  18b61dd5aae8     3  2017-12-22T16:30:15-0500 -15.332100  0.0008   
1885144  18b61dd5aae8     4  2017-12-22T16:30:20-0500 -14.292500  0.0065   
1885145  18b61dd5aae8     5  2017-12-22T16:30:25-0500 -16.090900  0.0087   
1885146  18b61dd5aae8     6  2017-12-22T16:30:30-0500 -16.971300  0.0206   
1885147  18b61dd5aae8     7  2017-12-22T16:30:35-0500 -15.477400  0.0124   
1885148  18b61dd5aae8     8  2017-12-22T16:30:40-0500 -18.168501  0.0660   
1885149  18b61dd5aae8     9  2017-12-22T16:30:45-0500 -12.208900  0.1289   

         awake  
1885140      1  
1885141      1  
1885142      1  
1885143      1  
1885144      1  
1885145      1  
1885146      1  
1885147      1  
1885148   

In [19]:
partitions = {}
for series in set(small_set_data["series_id"]):
    partitions[series] = []
    current_partition = pd.DataFrame()
    start_value = data_series[series]["awake"].iloc[0]
    for index, row  in data_series[series].iterrows():
        #print(row)
        if row["awake"] == start_value:
        # If the value is the same, append it to the current partition
            current_partition = pd.concat([current_partition, row.to_frame().T])
        else:
        # If the value is different, create a new partition
            partitions[series].append(current_partition)
            current_partition = pd.DataFrame()  # Reset the current partition
            current_partition = pd.concat([current_partition, row.to_frame().T])
            start_value = row["awake"]  # Update the start value

In [23]:
print(partitions['18b61dd5aae8'][2])

            series_id   step                 timestamp     anglez    enmo  \
1897237  18b61dd5aae8  12097  2017-12-23T09:18:05-0500  63.511398     0.0   
1897238  18b61dd5aae8  12098  2017-12-23T09:18:10-0500    55.4753  0.0057   
1897239  18b61dd5aae8  12099  2017-12-23T09:18:15-0500  54.178398  0.0084   
1897240  18b61dd5aae8  12100  2017-12-23T09:18:20-0500  62.499298     0.0   
1897241  18b61dd5aae8  12101  2017-12-23T09:18:25-0500  62.499298  0.0001   
...               ...    ...                       ...        ...     ...   
1906808  18b61dd5aae8  21668  2017-12-23T22:35:40-0500   -13.8579  0.0139   
1906809  18b61dd5aae8  21669  2017-12-23T22:35:45-0500   -13.6616  0.0139   
1906810  18b61dd5aae8  21670  2017-12-23T22:35:50-0500   -13.3994  0.0154   
1906811  18b61dd5aae8  21671  2017-12-23T22:35:55-0500   -13.5491  0.0207   
1906812  18b61dd5aae8  21672  2017-12-23T22:36:00-0500   -13.6627  0.0162   

        awake  
1897237     1  
1897238     1  
1897239     1  
1897240    

In [28]:
import os
# Assuming 'partitions' is your dictionary
for key, df_list in partitions.items():
    for index, df in enumerate(df_list):
        # Get the value of the first row of the "awake" column
        first_row_value = df["awake"].iloc[0]

        # Generate a unique file name based on key, index, and first row value
        file_name = f"{key}_index{index}_awake{first_row_value}.csv"
        
        # Define the path where the CSV file will be saved
        if first_row_value == 0:
            save_path = os.path.join("data/asleep", file_name)
        if first_row_value == 1:
            save_path = os.path.join("data/awake", file_name)

        # Save the DataFrame to the CSV file
        df.to_csv(save_path, index=False)